# sklearn-porter

Repository: https://github.com/nok/sklearn-porter

## LinearSVC

Documentation: [sklearn.svm.LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

### Loading data:

In [1]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X = iris_data.data
y = iris_data.target

print(X.shape, y.shape)

((150, 4), (150,))


### Train classifier:

In [2]:
from sklearn import svm

clf = svm.LinearSVC(C=1., random_state=0)
clf.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

### Transpile classifier:

In [4]:
%%time

from sklearn_porter import Porter

porter = Porter(clf, language='c')
output = porter.export()

print(output)

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

double coefficients[3][4] = {{0.184242094585, 0.451230000252, -0.807945877167, -0.450716600333}, {0.0528774557485, -0.892149952286, 0.403980844596, -0.937682166145}, {-0.850707843193, -0.986702149222, 1.38101044874, 1.86540956624}};
double intercepts[3] = {0.109562664067, 1.66367077767, -1.70961094165};

int predict (float features[]) {
    double class_val = -INFINITY;
    int class_idx = -1;
    int i, il, j, jl;
    for (i = 0, il = sizeof(coefficients) / sizeof (coefficients[0]); i < il; i++) {
        double prob = 0.;
        for (j = 0, jl = sizeof(coefficients[0]) / sizeof (coefficients[0][0]); j < jl; j++) {
            prob += coefficients[i][j] * features[j];
        }
        if (prob + intercepts[i] > class_val) {
            class_val = prob + intercepts[i];
            class_idx = i;
        }
    }
    return class_idx;
}

int main(int argc, const char * argv[]) {

    /* Features: */
    float features[argc-1];


### Run classification in C:

Save the transpiled estimator:

In [5]:
with open('linearsvc.c', 'w') as f:
    f.write(output)

Compiling:

In [6]:
%%bash

gcc linearsvc.c -std=c99 -lm -o linearsvc

Prediction:

In [7]:
%%bash

./linearsvc 1 2 3 4

2